In [ ]:
import pandas as pd
!pip install shap
!pip install pyngrok
!pip install dash
!pip install dash-bootstrap-components
!pip install plotly
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import shap
from pyngrok import ngrok

app = dash.Dash(__name__)
server = app.server

# materials with hardness values
materials = {
    "Steel": 1.0,
    "Aluminum": 0.6,
    "Titanium": 1.4,
    "Copper": 0.8,
    "Cast Iron": 1.2,
    "Brass": 0.7,
    "Bronze": 0.9,
    "Nickel": 1.5,
    "Cobalt": 1.3,
    "Stainless Steel": 1.6,
    "Tungsten Carbide": 2.0,
    "Inconel": 1.8,
    "Zinc": 0.5,
    "Magnesium": 0.4,
    "Platinum": 1.1
}

def generate_data(num_samples=1000):
    cutting_speeds = np.random.uniform(50, 500, num_samples)
    feed_rates = np.random.uniform(0.1, 2.0, num_samples)
    tool_geometry = np.random.uniform(0.5, 3.0, num_samples)
    environment_factors = np.random.uniform(1, 5, num_samples)
    machining_times = np.random.uniform(5, 60, num_samples)
    material_types = np.random.choice(list(materials.keys()), num_samples)

    tool_wear = [
        simulate_tool_wear(speed, feed, mat, geo, env, time)
        for speed, feed, mat, geo, env, time in zip(cutting_speeds, feed_rates, material_types, tool_geometry, environment_factors, machining_times)
    ]

    data = pd.DataFrame({
        'CuttingSpeed (m/min)': cutting_speeds,
        'FeedRate (mm/rev)': feed_rates,
        'MaterialType': material_types,
        'ToolGeometry (mm)': tool_geometry,
        'EnvironmentFactor': environment_factors,
        'MachiningTime (min)': machining_times,
        'ToolWear (mm)': tool_wear
    })
    data['MaterialHardness'] = data['MaterialType'].map(materials)
    return data

def simulate_tool_wear(speed, feed, material, geometry, env, time):
    base_wear = speed * 0.01 + feed * 0.5 + geometry * 0.3 + env * 0.1 + time * 0.05
    material_factor = materials[material]
    wear = base_wear * material_factor + np.random.normal(0, 0.15)
    return max(0, wear)

# Adjusted simulate_tool_wear_over_time function to prevent wear decrease
def simulate_tool_wear_over_time(speed, feed, geometry, environment, material, target_wear, machining_time, total_time=60, time_step=1):
    material_factor = materials[material]
    times = np.arange(0, total_time + time_step, time_step)
    wear_over_time = []

    last_wear = 0  # Ensure non-decreasing wear over time

    for t in times:
        wear = (speed * 0.01 + feed * 0.5 + geometry * 0.3 + environment * 0.1) * material_factor + t * 0.05 * material_factor
        wear = max(last_wear, wear + np.random.normal(0, 0.05))
        wear_over_time.append(wear)
        last_wear = wear  # Update last_wear to prevent decrease

    initial_wear = wear_over_time[0]
    wear_over_time = [w - initial_wear for w in wear_over_time]

    # Scale to align with target_wear at specified machining_time
    wear_at_machining_time = wear_over_time[int(machining_time / time_step)]
    if wear_at_machining_time != 0:
        scaling_factor = target_wear / wear_at_machining_time
        wear_over_time = [w * scaling_factor for w in wear_over_time]

    return times, wear_over_time

app.layout = html.Div([
    html.H1("Tool Wear Prediction App"),
    html.Div([
        html.Label("Cutting Speed (m/min)"),
        dcc.Input(id="cutting_speed", type="number", value=100, step=1),

        html.Label("Feed Rate (mm/rev)"),
        dcc.Input(id="feed_rate", type="number", value=1.0, step=0.1),

        html.Label("Tool Geometry (mm)"),
        dcc.Input(id="geometry", type="number", value=1.5, step=0.1),

        html.Label("Environment Factor"),
        dcc.Slider(id="environment_factor", min=1, max=5, step=0.5, value=3),

        html.Label("Machining Time (min)"),
        dcc.Input(id="machining_time", type="number", value=10, step=1),

        html.Label("Material Type"),
        dcc.Dropdown(
            id="material_type",
            options=[{"label": k, "value": k} for k in materials.keys()],
            value="Steel"
        ),

        html.Button("Predict Tool Wear", id="predict-button", n_clicks=0)
    ], style={'display': 'grid', 'gap': '10px', 'width': '50%', 'margin': 'auto'}),

    html.Hr(),

    html.Div(id="prediction-output"),
    html.Div(id="best-material-output", style={"font-weight": "bold"}),

    html.H2("Actual vs Predicted Tool Wear"),
    dcc.Graph(id="bar-chart"),

    html.H2("Feature Importance (SHAP)"),
    dcc.Graph(id="shap-plot"),

    html.H2("Tool Wear Over Time"),
    dcc.Graph(id="wear-over-time-plot")
])

@app.callback(
    Output("prediction-output", "children"),
    Output("best-material-output", "children"),
    Output("bar-chart", "figure"),
    Output("shap-plot", "figure"),
    Output("wear-over-time-plot", "figure"),
    [Input("cutting_speed", "value"),
     Input("feed_rate", "value"),
     Input("geometry", "value"),
     Input("environment_factor", "value"),
     Input("machining_time", "value"),
     Input("material_type", "value"),
     Input("predict-button", "n_clicks")]
)
def predict_tool_wear(speed, feed, geometry, environment, time, material, n_clicks):
    data = generate_data()
    X = pd.get_dummies(data.drop(columns=['ToolWear (mm)', 'MaterialType']), drop_first=True)
    y = data['ToolWear (mm)']
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(random_state=42, n_estimators=150, max_depth=20, min_samples_split=5)
    model.fit(X_train, y_train)

    explainer = shap.Explainer(model, X_train)

    input_data = pd.DataFrame({
        "CuttingSpeed (m/min)": [speed],
        "FeedRate (mm/rev)": [feed],
        "ToolGeometry (mm)": [geometry],
        "EnvironmentFactor": [environment],
        "MachiningTime (min)": [time],
        "MaterialHardness": [materials[material]]
    })
    input_data = pd.get_dummies(input_data).reindex(columns=X.columns, fill_value=0)
    input_scaled = scaler.transform(input_data)

    predicted_wear = model.predict(input_scaled)[0]

    best_material, min_wear = material, predicted_wear
    for mat, hardness in materials.items():
        test_data = input_data.copy()
        test_data['MaterialHardness'] = hardness
        test_scaled = scaler.transform(test_data)
        wear = model.predict(test_scaled)[0]
        if wear < min_wear:
            best_material, min_wear = mat, wear

    y_pred = model.predict(X_test)
    results_df = pd.DataFrame({
        'Actual': y_test,
        'Predicted': y_pred,
        'Material Type': data['MaterialType'].iloc[y_test.index]
    })
    summary_df = results_df.groupby('Material Type').mean().reset_index()
    bar_fig = px.bar(summary_df, x='Material Type', y=['Actual', 'Predicted'], barmode='group')

    shap_values = explainer(X_test[:min(50, len(X_test))])
    mean_shap_values = np.abs(shap_values.values).mean(axis=0)
    feature_importance_df = pd.DataFrame({
        "Feature": X.columns,
        "Importance": mean_shap_values
    }).sort_values(by="Importance", ascending=False)

    shap_fig = px.bar(feature_importance_df, x="Importance", y="Feature", orientation="h")
    shap_fig.update_layout(title="Feature Importance (SHAP)", yaxis=dict(autorange="reversed"))

    times, wear_over_time = simulate_tool_wear_over_time(
        speed, feed, geometry, environment, material, target_wear=predicted_wear, machining_time=time, total_time=60
    )
    time_plot = px.line(
        x=times, y=wear_over_time,
        labels={"x": "Time (minutes)", "y": "Tool Wear (mm)"},
        title="Tool Wear Progression Over Time"
    )

    return (
        f"Predicted Tool Wear: {predicted_wear:.2f} mm",
        f"Best Material for given parameters: {best_material} with estimated wear of {min_wear:.2f} mm",
        bar_fig,
        shap_fig,
        time_plot
    )

if __name__ == '__main__':
    app.run_server(debug=True)
ngrok.set_auth_token("2ofOD26EqMgz26JmqthP3fhDkXf_73iGHqRLCqfDu3qkE2MmP")
public_url = ngrok.connect(8050)
print(f"Public URL: {public_url}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.2
    Uninstalling Werkzeug-3.1.2:
      Successfully uninstalled Werkzeug-3.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 5.0 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

Public URL: NgrokTunnel: "https://6dbb-35-247-65-51.ngrok-free.app" -> "http://localhost:8050"
